In [2]:
!pip install elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 842.9 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 2.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [30]:
!pip install randomname

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 862.0 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for randomname: filename=randomname-0.2.1-py3-none-any.whl size=89194 sha256=ca5ab9340ccf86e01eb2c8437882f3ea6fb8c6f026a7da9d37473f1bdb288573
  Stored in directory: /home/jovyan/.cache/pip/wheels/99/b3/ae/c137ed34d7c385b74ae440b4f008183264ebe466ea0341db09
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=83c11bb9db3aa5aeb8a20765997326d012f76c8d9dcc7ca647172f1eff86e79e
  Stored in directory: /home/jovyan/.cache/pip/wheels/a7/ee/a5/19e91481be8bea594935d137578bfe77d6bf905e4595336f6b
Successfully built randomname fire


In [38]:
from elasticsearch import Elasticsearch, helpers
import yaml
import json
import bucketstore
import randomname


In [39]:
with open("secrets.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

bucketstore.login(
    access_key_id=cfg['spaces']['access'],
    secret_access_key=cfg['spaces']['secret'],
    region='nyc3',
    endpoint_url=cfg['spaces']['url']
)
bucket = bucketstore.get('wrathalake')

elastic = Elasticsearch(
    cloud_id=cfg['elastic']['cloudId'],
    api_key=(cfg['elastic']['apiKey'])
)

ridesKey = 'intermediate/rides.json'


In [40]:
rides = json.loads(bucket[ridesKey])


In [41]:
for ride in rides.keys():
    if rides[ride]['inElastic'] == False:
        print("Need to add data for ride {} into Elastic".format(ride))
        
        rideData = json.loads(bucket[rides[ride]['interFile']])
        rideName = '{}-{}'.format(rides[ride]['rideDate'], randomname.get_name())
        
        elasticData = [
            {
                '_op_type': 'index',
                '_index': 'equilab-rides',
                '_source': {
                    '@timestamp': point['time'],
                    'rideName': rideName,
                    'location': {
                        'lat': point['coords']['lat'],
                        'lon': point['coords']['long']
                    },
                    'elevation': point['elevation'],
                    'timeDelta': point.get('timeDelta', None),
                    'distance': point.get('distance', None),
                    'speed': point.get('speed', None),
                    'climb': point.get('climb', None),
                    'drop': point.get('drop', None)
                }
            }
            for point in rideData
        ]
        
        try:
            # make the bulk call, and get a response
            response = helpers.bulk(elastic, elasticData)

            #response = helpers.bulk(elastic, actions, index='employees', doc_type='people')
            print ("\nRESPONSE:", response)
        except Exception as e:
            print("\nERROR:", e)

Need to add data for ride raw/sources/equilab/training-2023-09-09.gpx into Elastic

RESPONSE: (6437, [])
Need to add data for ride raw/sources/equilab/training-2023-09-10.gpx into Elastic

RESPONSE: (6912, [])
Need to add data for ride raw/sources/equilab/training-2023-09-23.gpx into Elastic

RESPONSE: (9083, [])
Need to add data for ride raw/sources/equilab/training-2023-10-14.gpx into Elastic

RESPONSE: (14040, [])
Need to add data for ride raw/sources/equilab/training-2023-10-22.gpx into Elastic

RESPONSE: (10471, [])
